In [ ]:
import threading
import pickle as pkl
import networkx as nx
from networkx.readwrite import json_graph
import re, sys, csv, json, tqdm, random, pprint, pandas as pd
from community import generate_dendrogram, best_partition, partition_at_level

In [ ]:
def getEdgeWeight(a, b):
    global G
    try:
        temp= G[a][b]['weight']
        return temp
    except:
        return 0

In [ ]:
def loadTxt(name):
    return [i.strip() for i in open(f'../../data/{name}', 'r').readlines()]

restoredUsersList= set()
for i in loadTxt('unsuspended_8_Oct.txt'):
    restoredUsersList.add(i)

In [ ]:
suspendedUsersList= set()

with open(f'../../data/all_tweets_suspended_jan3_2021.pickle', 'rb') as f:
    try:
        while True:
            temp= pkl.load(f)
            for i in tqdm.tqdm(temp):
                suspendedUsersList.add(i['user']['id_str'])
    except:
        pass

In [ ]:
len(suspendedUsersList), len(restoredUsersList)

In [ ]:
def determineUser(id):
    global restoredUsersList, suspendedUsersList
    if id in restoredUsersList: return 'Restored'
    if id in suspendedUsersList: return 'Suspended'
    return 'Other'

In [ ]:
def getUserColor(typeUser):
    if typeUser=='Suspended': return '#ff0000'
    if typeUser=='Restored': return '#0000ff'
    return '#000000'

In [ ]:
def getEdgeColor(a, b):
    if a=='Suspended' and b=='Suspended': return '#ff0000'
    if a=='Restored' and b=='Restored': return '#0000ff'
    if (a=='Restored' and b=='Suspended') or (a=='Suspended' and b=='Restored'): return '#00ff00'
    return '#000000'

In [ ]:
D = {}
G = nx.DiGraph()

In [ ]:
nodes = {'Id': []} #, 'Label': []} #, 'Color': []}
edges = {'Source': [], 'Target': [], 'Weight': []} #, 'Color': []}

In [ ]:
with open(f'../../data/all_tweets_suspended_jan3_2021.pickle', 'rb') as f:
    try:
        while True:
            temp= pkl.load(f)
            for i in tqdm.tqdm(temp):
                if i.get('retweeted_status'):
                    suspended_user= i['user']['id_str']
                    retweeted_user= i['retweeted_status']['user']['id_str']
                    if suspended_user not in restoredUsersList: # Consider it as only suspended
                        G.add_edge(suspended_user, retweeted_user, weight= getEdgeWeight(suspended_user, retweeted_user) + 1)
#                         G.nodes[suspended_user]['type']= determineUser(suspended_user)
#                         G.nodes[retweeted_user]['type']= determineUser(retweeted_user)
    except EOFError:
        pass

In [ ]:
for i in G.nodes:
    nodes['Id'].append(i)
#     nodes['Label'].append(G.nodes[i]['type'])
#     nodes['Color'].append(getUserColor(G.nodes[i]['type']))

In [ ]:
for i in G.edges(data=True):
    edges['Source'].append(i[0])
    edges['Target'].append(i[1])
    edges['Weight'].append(i[2]['weight'])
#     edges['Color'].append(getEdgeColor(G.nodes[i[0]]['type'], G.nodes[i[1]]['type']))

In [ ]:
nodes= pd.DataFrame.from_dict(nodes)
edges= pd.DataFrame.from_dict(edges)

In [ ]:
print(nodes)
nodes.to_csv('./retweet-nodes-from-only-suspended.csv', index= False)

In [ ]:
print(edges)
edges.to_csv('./retweet-edges-from-only-suspended.csv', index= False)